In [1]:
# enter at past 10 candles 1h rsi 35 oversold, rsi_length 18, 15m cross k > d, 4h ema1 > ema3
# exit at 1h rsi 74 overbought
# stop loss 2 atr
# symbol ftm

In [2]:
import requests
import json
import pandas as pd
import numpy as np
import datetime as dt
import time
import ta
import os

In [3]:
# 調整 notebook 寬物
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }<style>"))

# 調整 output 不折疊
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [4]:
import winsound
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [5]:
from binance.client import Client
from dotenv import load_dotenv

# Get the path to the current directory
current_directory = os.getcwd()

# Specify the path to the .env file relative to the current directory
dotenv_path = os.path.join(current_directory, '.env')

# Load the environment variables from the .env file
load_dotenv(dotenv_path)

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)

     asset        free      locked
0      BTC  0.00000000  0.00000000
1      LTC  0.00000000  0.00000000
2      ETH  0.00000000  0.00000000
3      NEO  0.00000000  0.00000000
4      BNB  0.04622450  0.00000000
..     ...         ...         ...
536    EDU  0.00000000  0.00000000
537  WBETH  0.00000000  0.00000000
538    SUI  0.00000000  0.00000000
539   PEPE        0.00        0.00
540  FLOKI        0.00        0.00

[541 rows x 3 columns]


In [6]:
timezone = 8
symbol = 'ftmusdt'
interval = '15m'

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 3600
start_time = round(time.time() * 1000 - (86400 * 1000 * 5))
end_time = round(time.time() * 1000)

def define_time():
    
    global start_time, end_time
    
    # start epoch till now, use prior 5 days for this strategy
    start_time = round(time.time() * 1000 - (86400 * 1000 * 5))
    end_time = round(time.time() * 1000)

#     pre_start = pd.to_datetime(start_time, unit='ms') + pd.Timedelta(hours=timezone)
#     pre_start = pre_start.strftime('%Y-%m-%d %H:%M:%S')
#     pre_end = pd.to_datetime(end_time, unit='ms') + pd.Timedelta(hours=timezone)
#     pre_end = pre_end.strftime('%Y-%m-%d %H:%M:%S')
#     print(f'Start time: {pre_start} to {pre_end}')

In [7]:
dataframes = {}
# 先抓歷史資料好讓技術指標能成型
def create_raw(symbol, interval_arr, start_time, end_time, step):
    
    url = "https://api.binance.com/api/v3/klines"
    
    for interval in interval_arr:

        raw_df = pd.DataFrame()
        
        for timestamp in range(start_time, end_time, step):
            params = {"symbol": symbol.upper(),
                      "interval": interval,
                      "startTime": timestamp,
                      "endTime": timestamp + step}
            response = requests.get(url, params=params).json()
            out = pd.DataFrame(response, columns = ["Open_Time", "Open", "High", "Low", "Close",
                                                   "Volume", "Close_Time", "Quote asset volume",
                                                   "Number of trades", "Taker buy base asset volume",
                                                   "Taker buy quote asset volume", "Ignore"])
            raw_df = pd.concat([raw_df, out], axis = 0)

        raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

        dataframes[f'df_{interval}'] = raw_df


In [8]:
interval_arr = ['15m', '1h', '4h']
start_time_arr = [1683798299000]
ema_arr = [8, 18, 38]

In [9]:
sl_atr = 2
# tp_atr = 12
rsi_int = 18
rsi_up = 74
rsi_low = 35
kd_dir = 'Up'

In [10]:
def get_klines(symbol, interval, start_time, end_time):
    global dataframes
    df = dataframes[f'df_{interval}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [11]:
create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)

sample = dataframes[f'df_{interval_arr[0]}']
sample = sample[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)

sample['Open_Time'] = pd.to_datetime(sample['Open_Time'], unit='ms') + pd.Timedelta(hours=timezone)
sample['Open_Time'] = sample['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(sample)

               Open_Time    Open   Close    High     Low    Volume
0    2023-05-11 17:45:00  0.3695  0.3691  0.3697  0.3688  159873.0
1    2023-05-11 18:00:00  0.3690  0.3702  0.3702  0.3687  386284.0
2    2023-05-11 18:15:00  0.3703  0.3712  0.3713  0.3702  827769.0
3    2023-05-11 18:30:00  0.3711  0.3728  0.3731  0.3707  867200.0
4    2023-05-11 18:45:00  0.3727  0.3719  0.3734  0.3718  424543.0
..                   ...     ...     ...     ...     ...       ...
206  2023-05-28 21:15:00  0.3397  0.3401  0.3411  0.3387  640670.0
207  2023-05-28 21:30:00  0.3401  0.3412  0.3420  0.3400  903540.0
208  2023-05-28 21:45:00  0.3410  0.3404  0.3412  0.3403  307517.0
209  2023-05-28 22:00:00  0.3404  0.3396  0.3404  0.3393  287618.0
210  2023-05-28 22:15:00  0.3396  0.3388  0.3398  0.3384  247971.0

[1651 rows x 6 columns]


In [12]:
# raw_df = get_klines(symbol, '4h', start_time, end_time)
# ema = 8
# interval = '4h'
# column_name = f'ema_{ema}_{interval}'
# raw_df[column_name] = ta.trend.EMAIndicator(raw_df.Close, window=ema, fillna=True).ema_indicator()
# raw_df.to_csv(f'{ema}_{interval}_check_3.csv')

In [13]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down

In [14]:
def multi_timeframe():
    df = get_klines(symbol, interval_arr[0], start_time, end_time).copy()
    df = df.astype(float).round(4)

    for interval in interval_arr:
        raw_df = get_klines(symbol, interval, start_time, end_time)
        # ema
        for ema in ema_arr:
            column_name = f'ema_{ema}_{interval}'
            df[column_name] = ta.trend.EMAIndicator(raw_df.Close, window=ema, fillna=True).ema_indicator()

        # rsi
        rsi = ta.momentum.RSIIndicator(raw_df.Close, window = rsi_int)
        df[f'rsi_{interval}'] = rsi.rsi()

        # atr
        df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
        

    # reset index and set current index as a column
    df = df.reset_index()

    # set new index with integers
    df = df.set_index(pd.RangeIndex(len(df)))

    # time_format(timezone)
    df['Open_Time'] = df['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
    # fill up higher time frame empty values with equal interval between each value
    df = df.replace('', np.nan)
    df['ema_8_4h'] = df['ema_8_4h'].interpolate()
    df['ema_38_4h'] = df['ema_38_4h'].interpolate()
    df['rsi_1h'] = df['rsi_1h'].interpolate()
    
    return df

In [15]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):
# kd
    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 14, 3)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 14, 3)
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)

In [16]:
def conditions(df):
    # enter at past 10 candles 1h rsi 35 oversold, rsi_length 18, 15m cross k > d, 4h ema1 > ema3
    # exit at 1h rsi 74 overbought
    # stop loss 2 atr
    df['c1'] = False
    for i, row in df.iterrows():
        if i >= 10:
            if (df.loc[i-10:i-1, 'rsi_1h'] < rsi_low).any():
                df.loc[i, 'c1'] = True
                
        df['c2'] = df['kd_cross']
        
        df['c3'] = df[f'ema_{ema_arr[0]}_4h'] >= df[f'ema_{ema_arr[2]}_4h']

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3, 'signal'] = True
            
    # 下一根進場
    df['entry'] = False

In [17]:
def cancel_orders():
    
    open_orders = client.futures_get_open_orders()
    
    # Cancel all open orders
    if open_orders:
        # Cancel all open orders
        for order in open_orders:
            if order['positionSide'] == 'LONG':  
                cancel_response = client.futures_cancel_order(symbol=order['symbol'], orderId=order['orderId'])
                print(f"Canceled order: {order['symbol']} - {order['orderId']}")
    else:
        # Continue with the rest of the code
        print("No open LONG orders found.")
        time.sleep(1)  # Sleep for 1 second to avoid API rate limit

In [18]:
def enter_position(df):
        
    #-----Calculate entry price-----#
    close_val = df['Close']
    atr_val = df['atr']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = close_val.loc[close_val.index[-2]] - sl_atr * atr_val.loc[atr_val.index[-2]]
#     df.loc[df.index[-1], 'take_profit'] = close_val.loc[close_val.index[-2]] + tp_atr * atr_val.loc[atr_val.index[-2]]

    #-----position attributes-----#
    quantity = 1000
    entry_p = round(df.loc[df.index[-1], 'entry_p'], 4)
    stop_loss_p = round(df.loc[df.index[-1], 'stop_loss'], 4)
#     take_profit_p = round(df.loc[df.index[-1], 'take_profit'], 2)
    
    cancel_orders()

    try:
        order = client.futures_create_order(
            symbol=symbol.upper(),
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG',
        )
        print('Order created successfully.')       
            
    except Exception as e:
        print(f'Error creating order: {e}')
        
    print(str(symbol.upper()) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))

    return df

In [19]:
def check_sl(df, current_k):
    
    # 檢查前一根 Close 是否低於 SL，若低於，即刻停損
    
    last_signal_row = df.loc[df['signal'].eq(True) & df['Open_Time'].ne(current_k)].tail(1)
    if not last_signal_row.empty:
        current_sl = round(df.loc[last_signal_row.index[0], 'Close'] - df.loc[last_signal_row.index[0], 'atr'] * sl_atr, 4)
#         current_tp = round(df.loc[last_signal_row.index[0], 'Close'] + df.loc[last_signal_row.index[0], 'atr'] * tp_atr, 4)

    print(f'Current SL at {current_sl}')
    print(f'Current TP at DYNAMIC')
    
    # 確定停損
    if df.iloc[df.index[-2]]['Close'] <= current_sl:
        
        print('Last candle closed below SL, try create SL Order')
        
        cancel_orders()

        # 如果目前價格高於停損
        ticker = client.futures_symbol_ticker(symbol=symbol.upper())

        mark_price = round(float(ticker['price']), 4)
        
        stop_price = current_sl
        if current_sl > mark_price:
            stop_price = mark_price
        
        try:

            stop_loss_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='STOP_MARKET',
                stopPrice=stop_price,
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
        
            print(f'Stop loss created successfully at: {current_sl}')
            winsound.Beep(frequency, duration)
        
        except Exception as e:
            
            print(f'Error creating SL order: {e}')
          

In [20]:
def check_tp(df):
    global rsi_up
    
    try:
        # 隨時偵測出場條件是否成立，不必等 Close 發生
        if ((df.loc[df.index[-1], 'rsi_1h'] >= rsi_up)):
            take_profit_order = client.futures_create_order(
                symbol=symbol.upper(),
                side='SELL',
                type='TAKE_PROFIT_MARKET',
                stopPrice=df.loc[df.index[-1], 'Close'],
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
            winsound.Beep(frequency, duration)
            print('Take profit created successfully')
            
        else:
            print(f"Take profit target not reached rsi:{round(df.loc[df.index[-1], 'rsi_1h'], 4)}")
            
    except Exception as e:
        print(f'Error creating TP order: {e}')

In [21]:
import IPython

def restart_and_run_all():
    # Restart the kernel
    IPython.Application.instance().kernel.do_shutdown(True)

    # Re-run all code cells
    IPython.get_ipython().run_line_magic('run', '-i 000_015_LONG_kd15m_rsi1h_ema4h.ipynb')


In [22]:
from IPython.display import clear_output

line_count = 0
max_lines = 1
current_k = 0
restart_countdown = 2 * 60 * 60 # restart in 2 hours

def console_log(df):
    global line_count, max_lines, restart_countdown, loop_end_time
    try:
        df = df.reset_index(drop=True)
        df = df.round(4)
        print('----------------------------------------------------------------------------')
        print(f"{str(df.loc[df.index[-1], 'Open_Time'])} at {str(df.loc[df.index[-1], 'Close'])}")
        print()
        print(df[['Open_Time', 'Close', 'Volume', 'c1', 'c2', 'c3', 'signal', 'entry']].tail(5))
        print('----------------------------------------------------------------------------')
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Side':position['positionSide'],
                                            'Entry_P':round(float(position['entryPrice']),4),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unrealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                           }, index=[0])      
                print(position_df)
                print()

        # countdown for restarting session
        restart_countdown -= 1
        print(f'{restart_countdown} iterations until restart')

        if restart_countdown <= 100:
            restart_and_run_all()
        
        # count each loop time
        loop_end_time = time.time()
        print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")
        
    except Exception as e:
        print(f'Error UPDATING info: {e}')    

    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [23]:
def check_time(df):
    global current_k
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Open_Time'][len(df) - 1] != current_k:

            print('time changed')
            current_k = df['Open_Time'][len(df) - 1]

            if df.iloc[df.index[-2]]['signal'] == True:

                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'entry'] = True
                enter_position(df)

                print(df.tail(2))

    except Exception as e:
        print(f'Error creating ENTRY order: {e}')

In [24]:
loop_start_time = 0
loop_end_time = 0

def run():
    global current_k, restart_countdown, loop_start_time

    while True:
        loop_start_time = time.time()
        try:
            define_time()
            create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)
            df = multi_timeframe()
            indicators(df, kd_dir)

            conditions(df)
            check_time(df)

            # 檢查停損
            try:
                positions_info = client.futures_account()['positions']
                long_positions = [p for p in positions_info if p['positionSide'] == 'LONG' and float(p['positionAmt']) != 0 and p['symbol'] == symbol.upper()]

                if long_positions:
                    check_tp(df)
                    check_sl(df, current_k)

                else:
                    print('No LONG position. No SL')

            except Exception as e:
                print(f'Error checking SL: {e}')

            # 更新狀態
            console_log(df)
            
            time.sleep(2)
            
        except ConnectionError as e:
            print("Connection error occurred:", e)
            print("Retrying in 5 seconds...")
            time.sleep(5)

In [25]:
run()   

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

...

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
# c1 below 10 candles 1h rsi 35 oversold (rsi_length 18)
# c2 15m cross k > d (kd 14, 3)
# c3 4h ema1 > ema3
# exit at 1h rsi 74 overbought